In [ ]:
# default_exp data.token_classification


In [ ]:
# all_slow


In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# data.token_classification

> This module contains the bits required to use the fastai DataBlock API and/or mid-level data processing pipelines to organize your data for token classification tasks (e.g., Named entity recognition (NER), Part-of-speech tagging (POS), etc...)

In [ ]:
# export
import os
from typing import List, Tuple

from fastcore.all import *
from fastai.data.block import TransformBlock, Category, CategoryMap
from fastai.imports import *
from fastai.losses import CrossEntropyLossFlat
from fastai.torch_core import *
from fastai.torch_imports import *
from transformers import AutoModelForTokenClassification, logging, PretrainedConfig, PreTrainedTokenizerBase, PreTrainedModel

from blurr.utils import BLURR
from blurr.data.core import HF_BaseInput, HF_BeforeBatchTransform, first_blurr_tfm

logging.set_verbosity_error()


In [ ]:
# hide_input
import pdb

from datasets import load_dataset
from fastai.data.block import DataBlock, ColReader, ColSplitter
from fastai.data.core import DataLoader, DataLoaders, TfmdDL
from fastai.data.external import untar_data, URLs
from fastai.data.transforms import *
from fastcore.test import *
from nbverbose.showdoc import show_doc
from transformers import AutoTokenizer

from blurr.utils import print_versions
from blurr.data.core import HF_TextBlock

os.environ["TOKENIZERS_PARALLELISM"] = "false"
print("What we're running with at the time this documentation was generated:")
print_versions("torch fastai transformers")


What we're running with at the time this documentation was generated:
torch: 1.10.1+cu111
fastai: 2.5.3
transformers: 4.15.0


In [ ]:
# hide
# cuda
torch.cuda.set_device(1)
print(f"Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}")


Using GPU #1: GeForce GTX 1080 Ti


## Setup

We'll use a subset of `conll2003` to demonstrate how to configure your blurr code for token classification

In [ ]:
raw_datasets = load_dataset("conll2003")
raw_datasets


Reusing dataset conll2003 (/home/wgilliam/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6)


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

We need to get a list of the distinct entities we want to predict. If they are represented as list in their raw/readable form in another attribute/column in our dataset, we could use something like this to build a sorted list of distinct values as such: `labels = sorted(list(set([lbls for sublist in germ_eval_df.labels.tolist() for lbls in sublist])))`.

Fortunately, the `conll2003` dataset allows us to get at this list directly using the code below.

In [ ]:
labels = raw_datasets["train"].features["ner_tags"].feature.names
labels


['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [ ]:
#hide
print(raw_datasets["train"][0]["tokens"])
print(raw_datasets["train"][0]["ner_tags"])
print([(word, labels[label_idx]) for word, label_idx in zip(raw_datasets["train"][0]["tokens"], raw_datasets["train"][0]["ner_tags"])])


['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
[3, 0, 7, 0, 0, 0, 7, 0, 0]
[('EU', 'B-ORG'), ('rejects', 'O'), ('German', 'B-MISC'), ('call', 'O'), ('to', 'O'), ('boycott', 'O'), ('British', 'B-MISC'), ('lamb', 'O'), ('.', 'O')]


In [ ]:
conll2003_df = pd.DataFrame(raw_datasets["train"])


In [ ]:
model_cls = AutoModelForTokenClassification

pretrained_model_name = "roberta-base" # "bert-base-multilingual-cased"
n_labels = len(labels)

hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(
    pretrained_model_name, model_cls=model_cls, config_kwargs={"num_labels": n_labels}
)

hf_arch, type(hf_config), type(hf_tokenizer), type(hf_model)


('roberta',
 transformers.models.roberta.configuration_roberta.RobertaConfig,
 transformers.models.roberta.tokenization_roberta_fast.RobertaTokenizerFast,
 transformers.models.roberta.modeling_roberta.RobertaForTokenClassification)

## Utility classes and methods

### `get_slow_word_ids`

A method for getting fast tokenizer equivalent `word_ids` from ***any*** tokenizer so that Blurr users aren't restricted to using only fast tokenizers.

In [ ]:
# export
def get_slow_word_ids(hf_arch: str, hf_tokenizer: PreTrainedTokenizerBase, input_ids: List[int], special_tokens_mask):

    if hf_arch == "canine":
        toks = hf_tokenizer.convert_ids_to_tokens(input_ids, skip_special_tokens=True)
        word_list = [word for word in hf_tokenizer.convert_tokens_to_string([tok for tok in toks]).split()]
        n_tokens_per_word = [len(hf_tokenizer.tokenize(word)) + 1 for word in word_list]
    elif hf_arch == "deberta_v2":
        toks = hf_tokenizer.convert_ids_to_tokens(input_ids, skip_special_tokens=True)
        word_list = [word for word in hf_tokenizer.convert_tokens_to_string([tok for tok in toks]).split()]
        n_tokens_per_word = [len(hf_tokenizer.tokenize(word)) for word in word_list]
    else:
        word_list = hf_tokenizer.decode(input_ids, skip_special_tokens=True).split()
        n_tokens_per_word = [len(hf_tokenizer.tokenize(word)) for word in word_list]

    word_ids, word_idx, tok_idx = [], 0, 0
    while tok_idx < len(special_tokens_mask):
        if special_tokens_mask[tok_idx] == 1:
            word_ids.append(None)
            tok_idx += 1
        else:
            n_tokens = n_tokens_per_word[word_idx]
            word_ids += [word_idx] * n_tokens
            tok_idx += n_tokens
            word_idx += 1

    return word_ids


# def get_slow_word_ids(hf_tokenizer: PreTrainedTokenizerBase, text: Union[str, List[str]], input_ids: List[int], is_split_into_words: bool = False):
#     word_id_lists = [[idx] * len(hf_tokenizer.tokenize(str(word))) for idx, word in enumerate(text)]
#     word_ids = [word_id for word_id_list in word_id_lists for word_id in word_id_list]

#     encoding = hf_tokenizer(text, is_split_into_words=is_split_into_words, return_special_tokens_mask=True, padding="max_length", max_length=len(input_ids))
#     return [None if el == 1 else word_ids.pop(0) for el in encoding["special_tokens_mask"]]


In [ ]:
# TESTS for get_slow_word_ids()
test_arch, _, test_tokenizer, _ = BLURR.get_hf_objects(
    "xlm-mlm-en-2048", model_cls=AutoModelForTokenClassification, config_kwargs={"num_labels": len(labels)}
)

for idx in range(5):
    raw_word_list = conll2003_df.iloc[idx]["tokens"]
    raw_label_list = conll2003_df.iloc[idx]["ner_tags"]

    be = test_tokenizer(raw_word_list, is_split_into_words=True, return_special_tokens_mask=True)
    input_ids, spec_tok_mask = be["input_ids"], be["special_tokens_mask"]

    slow_word_ids = get_slow_word_ids(test_arch, test_tokenizer, input_ids, spec_tok_mask)
    if test_tokenizer.is_fast:
        test_eq(be.word_ids(), slow_word_ids)
    else:
        print(len(input_ids))
        print(len(slow_word_ids))


11
11
4
4
8
8
33
33
37
37


In [ ]:
show_doc(get_slow_word_ids)

<h4 id="get_slow_word_ids" class="doc_header"><code>get_slow_word_ids</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>get_slow_word_ids</code>(**`hf_arch`**:`str`, **`hf_tokenizer`**:`PreTrainedTokenizerBase`, **`input_ids`**:`List`\[`int`\], **`special_tokens_mask`**)



**Parameters:**


 - **`hf_arch`** : *`<class 'str'>`*

 - **`hf_tokenizer`** : *`<class 'transformers.tokenization_utils_base.PreTrainedTokenizerBase'>`*

 - **`input_ids`** : *`typing.List[int]`*

 - **`special_tokens_mask`** : *`<class 'inspect._empty'>`*


### `BaseLabelingStrategy` and implementations

Here we include a `BaseLabelingStrategy` abstract class and several different strategies for assigning labels to your tokenized inputs. The "only first token" and "BI" labeling strategies are discussed in the ["Token Classification"](https://huggingface.co/course/chapter7/2?fw=pt) section in part 7 of the Hugging Face's Transformers course.

In [ ]:
# export
class BaseLabelingStrategy:
    def __init__(self, hf_tokenizer: PreTrainedTokenizerBase, ignore_token_id: int = CrossEntropyLossFlat().ignore_index) -> None:
        self.hf_tokenizer = hf_tokenizer
        self.ignore_token_id = ignore_token_id

    def align_labels_with_tokens(self, label_names, word_ids, word_labels):
        raise NotImplementedError()


In [ ]:
# export
class OnlyFirstTokenLabelingStrategy(BaseLabelingStrategy):
    """
    Only the first token of word is associated with the label (all other subtokens with the `ignore_index_id`)
    """

    def align_labels_with_tokens(self, label_names, word_ids, word_labels):
        new_labels = []
        current_word = None
        for word_id in word_ids:
            if word_id != current_word:
                # start of a new word
                current_word = word_id
                label = self.ignore_token_id if word_id is None else word_labels[word_id]
                new_labels.append(label if isinstance(label, int) else label_names.index(label))
            else:
                # special token or another subtoken of current word
                new_labels.append(self.ignore_token_id)

        return new_labels


class SameLabelLabelingStrategy(BaseLabelingStrategy):
    """
    Every token associated with a given word is associated with the word's label
    """

    def align_labels_with_tokens(self, label_names, word_ids, word_labels):
        new_labels = []
        for word_id in word_ids:
            if word_id == None:
                new_labels.append(self.ignore_token_id)
            else:
                label = word_labels[word_id]
                new_labels.append(label if isinstance(label, int) else label_names.index(label))

        return new_labels


class BILabelingStrategy(BaseLabelingStrategy):
    """
    If using B/I labels, the first token assoicated to a given word gets the "B" label while all other tokens related
    to that same word get "I" labels.  If "I" labels don't exist, this strategy behaves like the `SameLabelLabelingStrategy`.
    """

    def align_labels_with_tokens(self, label_names, word_ids, word_labels):
        new_labels = []
        current_word = None
        for word_id in word_ids:
            if word_id != current_word:
                # start of a new word
                current_word = word_id
                label = self.ignore_token_id if word_id is None else word_labels[word_id]
                new_labels.append(label if isinstance(label, int) else label_names.index(label))
            elif word_id is None:
                # special token
                new_labels.append(self.ignore_token_id)
            else:
                # we're in the same word
                label = word_labels[word_id]
                label_name = label_names[label] if isinstance(label, int) else label

                # append the I-{ENTITY} if it exists in `labels`, else default to the `same_label` strategy
                iLabel = f"I-{label_name[2:]}"
                new_labels.append(label_names.index(iLabel) if iLabel in label_names else label)

        return new_labels


### Reconstructing inputs/labels

In [ ]:
# export
def align_labels_with_tokens(
    # A Hugging Face tokenizer
    hf_tokenizer: PreTrainedTokenizerBase,
    # List of input_ids for the tokens in a single piece of processed text
    input_ids: List[int],
    # List of label indexs for each token
    token_label_ids,
    # List of label names from witch the `label` indicies can be used to find the name of the label
    vocab,
    # The token ID that should be ignored when calculating the loss
    ignore_token_id=CrossEntropyLossFlat().ignore_index,
) -> List[Tuple[str, str]]:
    """
    Given a list of input IDs, the label ID associated to each, and the labels vocab, this method will return a list of tuples whereby
    each tuple defines the "token" and its label name. For example:
    [('ĠWay', B-PER), ('de', B-PER), ('ĠGill', I-PER), ('iam', I-PER), ('Ġloves'), ('ĠHug', B-ORG), ('ging', B-ORG), ('ĠFace', I-ORG)]
    """
    # convert ids to tokens
    toks = hf_tokenizer.convert_ids_to_tokens(input_ids)
    # align "tokens" with labels
    tok_labels = [
        (tok, "xUNKx" if label_id == ignore_token_id else vocab[label_id])
        for tok_id, tok, label_id in zip(input_ids, toks, token_label_ids)
        if tok_id not in hf_tokenizer.all_special_ids
    ]
    return tok_labels


In [ ]:
# TESTS for align_labels_with_tokens()
for idx in range(3):
    raw_word_list = conll2003_df.iloc[idx]['tokens']
    raw_label_list = conll2003_df.iloc[idx]['ner_tags']

    be = hf_tokenizer(raw_word_list, is_split_into_words=True)
    input_ids = be['input_ids']
    targ_ids = [-100 if (word_id == None) else raw_label_list[word_id] for word_id in be.word_ids()]

    tok_labels = align_labels_with_tokens(hf_tokenizer, input_ids, targ_ids, labels)

    for tok_label, targ_id in zip (tok_labels, [label_id for label_id in targ_ids if label_id != -100]):
        test_eq(tok_label[1], labels[targ_id])


In [ ]:
show_doc(align_labels_with_tokens)

<h4 id="align_labels_with_tokens" class="doc_header"><code>align_labels_with_tokens</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>align_labels_with_tokens</code>(**`hf_tokenizer`**:`PreTrainedTokenizerBase`, **`input_ids`**:`List`\[`int`\], **`token_label_ids`**, **`vocab`**, **`ignore_token_id`**=*`-100`*)

Given a list of input IDs, the label ID associated to each, and the labels vocab, this method will return a list of tuples whereby
each tuple defines the "token" and its label name. For example:
[('ĠWay', B-PER), ('de', B-PER), ('ĠGill', I-PER), ('iam', I-PER), ('Ġloves'), ('ĠHug', B-ORG), ('ging', B-ORG), ('ĠFace', I-ORG)]

**Parameters:**


 - **`hf_tokenizer`** : *`<class 'transformers.tokenization_utils_base.PreTrainedTokenizerBase'>`*	<p>A Hugging Face tokenizer</p>


 - **`input_ids`** : *`typing.List[int]`*	<p>List of input_ids for the tokens in a single piece of processed text</p>


 - **`token_label_ids`** : *`<class 'inspect._empty'>`*	<p>List of label indexs for each token</p>


 - **`vocab`** : *`<class 'inspect._empty'>`*	<p>List of label names from witch the `label` indicies can be used to find the name of the label</p>


 - **`ignore_token_id`** : *`<class 'int'>`*, *optional*	<p>The token ID that should be ignored when calculating the loss</p>



**Returns**:
	
 * *`typing.List[typing.Tuple[str, str]]`*

In [ ]:
# export
def align_labels_with_words(
    hf_arch: str, 
    # A Hugging Face tokenizer
    hf_tokenizer: PreTrainedTokenizerBase,
    # A list of tuples, where each represents a token and its label (e.g., [('ĠHug', B-ORG), ('ging', B-ORG), ('ĠFace', I-ORG), ...])
    tok_labels
) -> List[Tuple[str, str]]:
    """
    Given a list of tuples where each tuple defines a token and its label, return a list of tuples whereby each tuple defines the
    "word" and its label. Method assumes that model inputs are a list of words, and in conjunction with the `align_labels_with_tokens` method,
    allows the user to reconstruct the orginal raw inputs and labels.
    """
    # recreate raw words list (we assume for token classification that the input is a list of words)
    words = hf_tokenizer.convert_tokens_to_string([tok_label[0] for tok_label in tok_labels]).split()

    if hf_arch == "canine":
        word_list = [f"{word} " for word in words]
    else:
        word_list = [word for word in words]
    
    # align "words" with labels
    word_labels, idx = [], 0
    for word in word_list:
        word_labels.append((word, tok_labels[idx][1]))
        idx += len(hf_tokenizer.tokenize(word))

    return word_labels


In [ ]:
# TESTS for align_labels_with_words()
for idx in range(5):
    raw_word_list = conll2003_df.iloc[idx]['tokens']
    raw_label_list = conll2003_df.iloc[idx]['ner_tags']

    be = hf_tokenizer(raw_word_list, is_split_into_words=True)
    input_ids = be['input_ids']
    targ_ids = [-100 if (word_id == None) else raw_label_list[word_id] for word_id in be.word_ids()]

    tok_labels = align_labels_with_tokens(hf_tokenizer, input_ids, targ_ids, labels)
    word_labels = align_labels_with_words(hf_arch, hf_tokenizer, tok_labels)

    for word_label, raw_word, raw_label_id in zip (word_labels, raw_word_list, raw_label_list):
        test_eq(word_label[0], raw_word)
        test_eq(word_label[1], labels[raw_label_id])

In [ ]:
show_doc(align_labels_with_words)

<h4 id="align_labels_with_words" class="doc_header"><code>align_labels_with_words</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>align_labels_with_words</code>(**`hf_arch`**:`str`, **`hf_tokenizer`**:`PreTrainedTokenizerBase`, **`tok_labels`**)

Given a list of tuples where each tuple defines a token and its label, return a list of tuples whereby each tuple defines the
"word" and its label. Method assumes that model inputs are a list of words, and in conjunction with the [`align_labels_with_tokens`](/blurr/data-token-classification.html#align_labels_with_tokens) method,
allows the user to reconstruct the orginal raw inputs and labels.

**Parameters:**


 - **`hf_arch`** : *`<class 'str'>`*

 - **`hf_tokenizer`** : *`<class 'transformers.tokenization_utils_base.PreTrainedTokenizerBase'>`*	<p>A Hugging Face tokenizer</p>


 - **`tok_labels`** : *`<class 'inspect._empty'>`*	<p>A list of tuples, where each represents a token and its label (e.g., [('ĠHug', B-ORG), ('ging', B-ORG), ('ĠFace', I-ORG), ...])</p>



**Returns**:
	
 * *`typing.List[typing.Tuple[str, str]]`*

## Preprocessing

In [ ]:
# export
def pre_process_token_classification(
    # Your pd.DataFrame
    raw_df,
    hf_arch: str,
    # A Hugging Face tokenizer
    hf_tokenizer: PreTrainedTokenizerBase,
    # The token ID that should be ignored when calculating the loss
    ignore_token_id=CrossEntropyLossFlat().ignore_index,
    # The label names (if not specified, will build from DataFrame)
    label_names: Optional[List[str]] = None,
    # The attribute in your dataset that contains the list of words (default: 'tokens')
    word_list_attr: str = "tokens",
    # The attribute in your dataset that contains the list of labels associated to each word (default: 'labels')
    label_list_attr: str = "labels",
    # The labeling strategy you want to apply when associating labels with word tokens
    labeling_strategy_cls: BaseLabelingStrategy = OnlyFirstTokenLabelingStrategy,
    # Other column data from the raw DataFrame you want to include in the processed DataFrame
    keep_cols: list = [],
    # Any keyword arguments you want your Hugging Face tokenizer to use during tokenization
    tok_kwargs: dict = {"is_split_into_words": True, "return_special_tokens_mask": True},
) -> Tuple[pd.DataFrame, List[str]]:
    """
    This preprocessing routine is designed to work with labels that are a list of label names (e.g., B-PER, I-PER, etc...) or
    a list of label Ids (e.g., 0, 0, 2, etc...) which can be indexed into `label_names` to get their names.
    Assuming you are using the later approach, you must pass a list of label names into this method via the `label_names` argument.
    If you are using the former approach, passing `label_names` is optional. If you don't, asorted list of the the distinct label
    names will be created for you, which you should specify in either the mid or low-level API to ensure the model will use
    the right label indexes when building your targets.
    """
    df = raw_df.copy()
    labeling_strategy = labeling_strategy_cls(hf_tokenizer=hf_tokenizer, ignore_token_id=ignore_token_id)

    if label_names is None:
        label_names = sorted(list(set([lbls for sublist in df[label_list_attr].tolist() for lbls in sublist])))

    proc_data = []
    for row_idx, row in df.iterrows():
        # fetch data elements required to build a modelable dataset
        words, word_labels = row[word_list_attr], row[label_list_attr]

        if not is_listy(words):
            words = words.split()

        encoding = hf_tokenizer(words, truncation=True, **tok_kwargs)
        word_ids = (
            encoding.word_ids()
            if hf_tokenizer.is_fast
            else get_slow_word_ids(hf_arch, hf_tokenizer, encoding["input_ids"], encoding["special_tokens_mask"])
        )
        aligned_labels = labeling_strategy.align_labels_with_tokens(label_names, word_ids, word_labels)

        row_data = [encoding["input_ids"], aligned_labels, words, word_labels]
        row_data += [row[col] for col in keep_cols]
        proc_data.append(row_data)

    # put processed data into a new DataFrame and return
    proc_df = pd.DataFrame(proc_data, columns=["input_ids", "label_ids", word_list_attr, label_list_attr] + keep_cols)
    return proc_df, label_names


In [ ]:
show_doc(pre_process_token_classification)


<h4 id="pre_process_token_classification" class="doc_header"><code>pre_process_token_classification</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>pre_process_token_classification</code>(**`raw_df`**, **`hf_arch`**:`str`, **`hf_tokenizer`**:`PreTrainedTokenizerBase`, **`ignore_token_id`**=*`-100`*, **`label_names`**:`Optional`\[`List`\[`str`\]\]=*`None`*, **`word_list_attr`**:`str`=*`'tokens'`*, **`label_list_attr`**:`str`=*`'labels'`*, **`labeling_strategy_cls`**:[`BaseLabelingStrategy`](/blurr/data-token-classification.html#BaseLabelingStrategy)=*`OnlyFirstTokenLabelingStrategy`*, **`keep_cols`**:`list`=*`[]`*, **`tok_kwargs`**:`dict`=*`{'is_split_into_words': True, 'return_special_tokens_mask': True}`*)

This preprocessing routine is designed to work with labels that are a list of label names (e.g., B-PER, I-PER, etc...) or
a list of label Ids (e.g., 0, 0, 2, etc...) which can be indexed into `label_names` to get their names.
Assuming you are using the later approach, you must pass a list of label names into this method via the `label_names` argument.
If you are using the former approach, passing `label_names` is optional. If you don't, asorted list of the the distinct label
names will be created for you, which you should specify in either the mid or low-level API to ensure the model will use
the right label indexes when building your targets.

**Parameters:**


 - **`raw_df`** : *`<class 'inspect._empty'>`*	<p>Your pd.DataFrame</p>


 - **`hf_arch`** : *`<class 'str'>`*

 - **`hf_tokenizer`** : *`<class 'transformers.tokenization_utils_base.PreTrainedTokenizerBase'>`*	<p>A Hugging Face tokenizer</p>


 - **`ignore_token_id`** : *`<class 'int'>`*, *optional*	<p>The token ID that should be ignored when calculating the loss</p>


 - **`label_names`** : *`typing.Optional[typing.List[str]]`*, *optional*	<p>The label names (if not specified, will build from DataFrame)</p>


 - **`word_list_attr`** : *`<class 'str'>`*, *optional*	<p>The attribute in your dataset that contains the list of words (default: 'tokens')</p>


 - **`label_list_attr`** : *`<class 'str'>`*, *optional*	<p>The attribute in your dataset that contains the list of labels associated to each word (default: 'labels')</p>


 - **`labeling_strategy_cls`** : *`<class '__main__.BaseLabelingStrategy'>`*, *optional*	<p>The labeling strategy you want to apply when associating labels with word tokens</p>


 - **`keep_cols`** : *`<class 'list'>`*, *optional*	<p>Other column data from the raw DataFrame you want to include in the processed DataFrame</p>


 - **`tok_kwargs`** : *`<class 'dict'>`*, *optional*	<p>Any keyword arguments you want your Hugging Face tokenizer to use during tokenization</p>



**Returns**:
	
 * *`typing.Tuple[pandas.core.frame.DataFrame, typing.List[str]]`*

How to preprocess your data (labels are Ids)

In [ ]:
proc_df, ds_labels = pre_process_token_classification(
    conll2003_df, hf_arch, hf_tokenizer, label_names=labels, word_list_attr="tokens", label_list_attr="ner_tags", labeling_strategy_cls=OnlyFirstTokenLabelingStrategy
)

print(len(proc_df))
print(ds_labels)
proc_df.head()


14041
['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


,input_ids,label_ids,tokens,ner_tags
0,"[0, 1281, 24020, 1859, 486, 7, 13978, 1089, 17988, 479, 2]","[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100]","[EU, rejects, German, call, to, boycott, British, lamb, .]","[3, 0, 7, 0, 0, 0, 7, 0, 0]"
1,"[0, 2155, 20809, 2]","[-100, 1, 2, -100]","[Peter, Blackburn]","[1, 2]"
2,"[0, 6823, 16551, 16416, 8008, 12, 3669, 12, 2036, 2]","[-100, 5, -100, -100, 0, -100, -100, -100, -100, -100]","[BRUSSELS, 1996-08-22]","[5, 0]"
3,"[0, 20, 796, 1463, 26, 15, 296, 24, 19286, 19, 1859, 2949, 7, 2360, 7, 23795, 1089, 17988, 454, 4211, 3094, 549, 7758, 12094, 2199, 64, 28, 20579, 7, 14336, 479, 2]","[-100, 0, 3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]","[The, European, Commission, said, on, Thursday, it, disagreed, with, German, advice, to, consumers, to, shun, British, lamb, until, scientists, determine, whether, mad, cow, disease, can, be, transmitted, to, sheep, .]","[0, 3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,"[0, 1600, 128, 29, 4915, 7, 5, 796, 1332, 128, 29, 24443, 1540, 26978, 525, 5577, 4621, 26, 15, 307, 2360, 197, 907, 14336, 38542, 31, 749, 97, 87, 1444, 454, 5, 6441, 2949, 21, 18618, 479, 2]","[-100, 5, 0, -100, 0, 0, 0, 3, 4, 0, -100, 0, 0, 1, 2, -100, -100, 0, 0, 0, 0, 0, 0, 0, -100, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, -100]","[Germany, 's, representative, to, the, European, Union, 's, veterinary, committee, Werner, Zwingmann, said, on, Wednesday, consumers, should, buy, sheepmeat, from, countries, other, than, Britain, until, the, scientific, advice, was, clearer, .]","[5, 0, 0, 0, 0, 3, 4, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0]"


How to preprocess your data (labels are names)

In [ ]:
conll2003_labeled_df = conll2003_df.copy()
conll2003_labeled_df.ner_tags = conll2003_labeled_df.ner_tags.apply(lambda v: [labels[lbl_id] for lbl_id in v])
conll2003_labeled_df.head(2)

,chunk_tags,id,ner_tags,pos_tags,tokens
0,"[11, 21, 11, 12, 21, 22, 11, 12, 0]",0,"[B-ORG, O, B-MISC, O, O, O, B-MISC, O, O]","[22, 42, 16, 21, 35, 37, 16, 21, 7]","[EU, rejects, German, call, to, boycott, British, lamb, .]"
1,"[11, 12]",1,"[B-PER, I-PER]","[22, 22]","[Peter, Blackburn]"


In [ ]:
proc_df, ds_labels = pre_process_token_classification(
    conll2003_labeled_df, hf_arch, hf_tokenizer, label_names=labels, word_list_attr="tokens", label_list_attr="ner_tags", labeling_strategy_cls=OnlyFirstTokenLabelingStrategy
)

print(len(proc_df))
print(ds_labels)
proc_df.head()

14041
['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


,input_ids,label_ids,tokens,ner_tags
0,"[0, 1281, 24020, 1859, 486, 7, 13978, 1089, 17988, 479, 2]","[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100]","[EU, rejects, German, call, to, boycott, British, lamb, .]","[B-ORG, O, B-MISC, O, O, O, B-MISC, O, O]"
1,"[0, 2155, 20809, 2]","[-100, 1, 2, -100]","[Peter, Blackburn]","[B-PER, I-PER]"
2,"[0, 6823, 16551, 16416, 8008, 12, 3669, 12, 2036, 2]","[-100, 5, -100, -100, 0, -100, -100, -100, -100, -100]","[BRUSSELS, 1996-08-22]","[B-LOC, O]"
3,"[0, 20, 796, 1463, 26, 15, 296, 24, 19286, 19, 1859, 2949, 7, 2360, 7, 23795, 1089, 17988, 454, 4211, 3094, 549, 7758, 12094, 2199, 64, 28, 20579, 7, 14336, 479, 2]","[-100, 0, 3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]","[The, European, Commission, said, on, Thursday, it, disagreed, with, German, advice, to, consumers, to, shun, British, lamb, until, scientists, determine, whether, mad, cow, disease, can, be, transmitted, to, sheep, .]","[O, B-ORG, I-ORG, O, O, O, O, O, O, B-MISC, O, O, O, O, O, B-MISC, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
4,"[0, 1600, 128, 29, 4915, 7, 5, 796, 1332, 128, 29, 24443, 1540, 26978, 525, 5577, 4621, 26, 15, 307, 2360, 197, 907, 14336, 38542, 31, 749, 97, 87, 1444, 454, 5, 6441, 2949, 21, 18618, 479, 2]","[-100, 5, 0, -100, 0, 0, 0, 3, 4, 0, -100, 0, 0, 1, 2, -100, -100, 0, 0, 0, 0, 0, 0, 0, -100, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, -100]","[Germany, 's, representative, to, the, European, Union, 's, veterinary, committee, Werner, Zwingmann, said, on, Wednesday, consumers, should, buy, sheepmeat, from, countries, other, than, Britain, until, the, scientific, advice, was, clearer, .]","[B-LOC, O, O, O, O, B-ORG, I-ORG, O, O, O, B-PER, I-PER, O, O, O, O, O, O, O, O, O, O, O, B-LOC, O, O, O, O, O, O, O]"


## Mid-level API

### Targets

#### `HF_TokenTensorCategory`

In [ ]:
# export
class HF_TokenTensorCategory(TensorBase):
    pass


#### `HF_TokenCategorize`

`HF_TokenCategorize` modifies the fastai `Categorize` transform in a couple of ways.  

First, it allows your targets to consist of a `Category` ***per*** token, and second, it uses the idea of an `ignore_token_id` to mask subtokens that don't need a prediction.  For example, the target of special tokens (e.g., pad, cls, sep) are set to `ignore_token_id` as are subsequent sub-tokens of a given token should more than 1 sub-token make it up.

In [ ]:
# export
class HF_TokenCategorize(Transform):
    """Reversible transform of a list of category string to `vocab` id"""

    def __init__(
        self,
        # The unique list of entities (e.g., B-LOC) (default: CategoryMap(vocab))
        vocab=None,
        # The token used to identifiy ignored tokens (default: xIGNx)
        ignore_token=None,
        # The token ID that should be ignored when calculating the loss (default: CrossEntropyLossFlat().ignore_index)
        ignore_token_id=None,
    ):
        self.vocab = None if vocab is None else CategoryMap(vocab, sort=False)
        self.ignore_token = "[xIGNx]" if ignore_token is None else ignore_token
        self.ignore_token_id = CrossEntropyLossFlat().ignore_index if ignore_token_id is None else ignore_token_id

        self.loss_func, self.order = CrossEntropyLossFlat(ignore_index=self.ignore_token_id), 1

    def setups(self, dsets):
        if self.vocab is None and dsets is not None:
            self.vocab = CategoryMap(dsets)
        self.c = len(self.vocab)

    def encodes(self, labels):
        # if `val` is the label name (e.g., B-PER, I-PER, etc...), lookup the corresponding index in the vocab using
        # `self.vocab.o2i`
        ids = [val if (isinstance(val, int)) else self.vocab.o2i[val] for val in labels]
        return HF_TokenTensorCategory(ids)

    def decodes(self, encoded_labels):
        return Category([(self.vocab[lbl_id]) for lbl_id in encoded_labels if lbl_id != self.ignore_token_id])


#### `HF_TokenCategoryBlock`

In [ ]:
# export
def HF_TokenCategoryBlock(
    # The unique list of entities (e.g., B-LOC) (default: CategoryMap(vocab))
    vocab=None,
    # The token used to identifiy ignored tokens (default: xIGNx)
    ignore_token=None,
    # The token ID that should be ignored when calculating the loss (default: CrossEntropyLossFlat().ignore_index)
    ignore_token_id=None,
):
    """`TransformBlock` for per-token categorical targets"""
    return TransformBlock(type_tfms=HF_TokenCategorize(vocab=vocab, ignore_token=ignore_token, ignore_token_id=ignore_token_id))


In [ ]:
show_doc(HF_TokenCategoryBlock)


<h4 id="HF_TokenCategoryBlock" class="doc_header"><code>HF_TokenCategoryBlock</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>HF_TokenCategoryBlock</code>(**`vocab`**=*`None`*, **`ignore_token`**=*`None`*, **`ignore_token_id`**=*`None`*)

`TransformBlock` for per-token categorical targets

**Parameters:**


 - **`vocab`** : *`<class 'NoneType'>`*, *optional*	<p>The unique list of entities (e.g., B-LOC) (default: CategoryMap(vocab))</p>


 - **`ignore_token`** : *`<class 'NoneType'>`*, *optional*	<p>The token used to identifiy ignored tokens (default: xIGNx)</p>


 - **`ignore_token_id`** : *`<class 'NoneType'>`*, *optional*	<p>The token ID that should be ignored when calculating the loss (default: CrossEntropyLossFlat().ignore_index)</p>



### Inputs

#### `HF_TokenClassInput`

Again, we define a custom class, `HF_TokenClassInput`, for the @typedispatched methods to use so that we can override how token classification inputs/targets are assembled, as well as, how the data is shown via methods like `show_batch` and `show_results`.

In [ ]:
# export
class HF_TokenClassInput(HF_BaseInput):
    pass


#### `HF_TokenClassBeforeBatchTransform`

`HF_TokenClassBeforeBatchTransform` is used to exclude any of the target's tokens we don't want to include in the loss calcuation (e.g. padding, cls, sep, etc...). Notice also that we default `is_split_into_words = True` since for most token classification problems, each example comes in the form as a list of words and a list of entity labels.

In [ ]:
# export
class HF_TokenClassBeforeBatchTransform(HF_BeforeBatchTransform):
    def __init__(
        self,
        # The abbreviation/name of your Hugging Face transformer architecture (e.b., bert, bart, etc..)
        hf_arch: str,
        # A specific configuration instance you want to use
        hf_config: PretrainedConfig,
        # A Hugging Face tokenizer
        hf_tokenizer: PreTrainedTokenizerBase,
        # A Hugging Face model
        hf_model: PreTrainedModel,
        # If you are passing in the "inpu_ids" as your inputs, set `is_numericalised` = True
        is_numericalized: bool = False,
        # The token ID that should be ignored when calculating the loss
        ignore_token_id=CrossEntropyLossFlat().ignore_index,
        # The labeling strategy you want to apply when associating labels with word tokens
        labeling_strategy_cls: BaseLabelingStrategy = OnlyFirstTokenLabelingStrategy,
        # To control the length of the padding/truncation. It can be an integer or None,
        # in which case it will default to the maximum length the model can accept. If the model has no
        # specific maximum input length, truncation/padding to max_length is deactivated.
        # See [Everything you always wanted to know about padding and truncation](https://huggingface.co/transformers/preprocessing.html#everything-you-always-wanted-to-know-about-padding-and-truncation)
        max_length: int = None,
        # To control the `padding` applied to your `hf_tokenizer` during tokenization. If None, will default to
        # `False` or `'do_not_pad'.
        # See [Everything you always wanted to know about padding and truncation](https://huggingface.co/transformers/preprocessing.html#everything-you-always-wanted-to-know-about-padding-and-truncation)
        padding: Union[bool, str] = True,
        # To control `truncation` applied to your `hf_tokenizer` during tokenization. If None, will default to
        # `False` or `do_not_truncate`.
        # See [Everything you always wanted to know about padding and truncation](https://huggingface.co/transformers/preprocessing.html#everything-you-always-wanted-to-know-about-padding-and-truncation)
        truncation: Union[bool, str] = True,
        # The `is_split_into_words` argument applied to your `hf_tokenizer` during tokenization. Set this to `True`
        # if your inputs are pre-tokenized (not numericalized)
        is_split_into_words: bool = True,
        # Any other keyword arguments you want included when using your `hf_tokenizer` to tokenize your inputs
        tok_kwargs={},
        # Keyword arguments to apply to `HF_TokenClassBeforeBatchTransform`
        **kwargs
    ):
        tok_kwargs = {**tok_kwargs, **{"return_special_tokens_mask": True}}

        super().__init__(
            hf_arch,
            hf_config,
            hf_tokenizer,
            hf_model,
            is_numericalized=is_numericalized,
            ignore_token_id=ignore_token_id,
            max_length=max_length,
            padding=padding,
            truncation=truncation,
            is_split_into_words=is_split_into_words,
            tok_kwargs=tok_kwargs,
            **kwargs
        )

        self.labeling_strategy = labeling_strategy_cls(hf_tokenizer, ignore_token_id=ignore_token_id)

    def encodes(self, samples):
        encoded_samples, batch_encoding = super().encodes(samples, return_batch_encoding=True)

        # if there are no targets (e.g., when used for inference) or the inputs/targets have already been preprocessed, 
        # there is no need to do any post-processing on the labels
        if len(encoded_samples[0]) == 1 or self.is_numericalized:
            return encoded_samples

        # get the type of our targets (by default will be HF_TokenTensorCategory)
        target_cls = type(encoded_samples[0][1])

        # we assume that first target = the categories we want to predict for each token
        updated_samples = []
        for idx, s in enumerate(encoded_samples):
            word_ids = (
                batch_encoding.word_ids(idx)
                if self.hf_tokenizer.is_fast
                else get_slow_word_ids(self.hf_arch, self.hf_tokenizer, s[0]["input_ids"], s[0]["special_tokens_mask"])
            )
            targ_ids = target_cls(self.labeling_strategy.align_labels_with_tokens(None, word_ids, s[1].tolist()))
            
            updated_samples.append((s[0], targ_ids))

        return updated_samples


### How to use

In [ ]:
before_batch_tfm = HF_TokenClassBeforeBatchTransform(hf_arch, hf_config, hf_tokenizer, hf_model)
blocks = (HF_TextBlock(before_batch_tfm=before_batch_tfm, input_return_type=HF_TokenClassInput), HF_TokenCategoryBlock(vocab=labels))

dblock = DataBlock(blocks=blocks, get_x=ColReader("tokens"), get_y=ColReader("ner_tags"), splitter=RandomSplitter())


In [ ]:
# hide
# dblock.summary(conll2003_df)


In [ ]:
dls = dblock.dataloaders(conll2003_df, bs=4)


In [ ]:
b = dls.one_batch()


In [ ]:
len(b), b[0]["input_ids"].shape, b[1].shape


(2, torch.Size([4, 72]), torch.Size([4, 72]))

In [ ]:
# export
@typedispatch
def show_batch(
    # This typedispatched `show_batch` will be called for `HF_TokenClassInput` typed inputs
    x: HF_TokenClassInput,
    y,
    # Your raw inputs/targets
    samples,
    # Your `DataLoaders`. This is required so as to get at the Hugging Face objects for
    # decoding them into something understandable
    dataloaders,
    # Your `show_batch` context
    ctxs=None,
    # The maximum number of items to show
    max_n=6,
    # Any truncation your want applied to your decoded inputs
    trunc_at=None,
    # Any other keyword arguments you want applied to `show_batch`
    **kwargs,
):
    # grab our tokenizer
    tfm = first_blurr_tfm(dataloaders, before_batch_tfm_class=HF_TokenClassBeforeBatchTransform)
    hf_arch, hf_tokenizer = tfm.hf_arch, tfm.hf_tokenizer
    vocab = dataloaders.vocab

    res = L()
    for inp, trg, sample in zip(x, y, samples):
        # align "tokens" with labels
        tok_labels = align_labels_with_tokens(hf_tokenizer, inp, trg, vocab)
        # align "words" with labels
        word_labels = align_labels_with_words(hf_arch, hf_tokenizer, tok_labels)
        # stringify list of (word,label) for example
        res.append([f"{[ word_targ for idx, word_targ in enumerate(word_labels) if (trunc_at is None or idx < trunc_at) ]}"])

    display_df(pd.DataFrame(res, columns=["word / target label"])[:max_n])
    return ctxs


In [ ]:
dls.show_batch(dataloaders=dls, max_n=5, trunc_at=20)


,word / target label
0,"[('Compared', 'O'), ('with', 'O'), ('the', 'O'), ('end', 'O'), ('of', 'O'), ('last', 'O'), ('year', 'O'), (',', 'O'), ('when', 'O'), ('T&N', 'B-ORG'), ('predicted', 'O'), ('a', 'O'), ('sluggish', 'O'), ('first', 'O'), ('half', 'O'), ('and', 'O'), ('a', 'O'), ('rebound', 'O'), ('later', 'O'), ('in', 'O')]"
1,"[('Captain', 'O'), ('Firmin', 'B-PER'), ('Gatera', 'I-PER'), (',', 'O'), ('spokesman', 'O'), ('for', 'O'), ('the', 'O'), ('Tutsi-dominated', 'B-MISC'), ('Rwandan', 'B-MISC'), ('army', 'O'), (',', 'O'), ('told', 'O'), ('Reuters', 'B-ORG'), ('in', 'O'), ('Kigali', 'B-LOC'), ('that', 'O'), ('17', 'O'), ('of', 'O'), ('the', 'O'), ('28', 'O')]"
2,"[('""', 'O'), ('The', 'O'), ('ultimatum', 'O'), ('(', 'O'), ('to', 'O'), ('storm', 'O'), ('Grozny', 'B-LOC'), (')', 'O'), ('is', 'O'), ('no', 'O'), ('longer', 'O'), ('an', 'O'), ('issue', 'O'), (',', 'O'), ('""', 'O'), ('he', 'O'), ('said', 'O'), ('quoting', 'O'), ('Ischinger', 'B-PER'), (',', 'O')]"
3,"[('Hershiser', 'B-PER'), ('(', 'O'), ('14-7', 'O'), (')', 'O'), (',', 'O'), ('who', 'O'), ('allowed', 'O'), ('three', 'O'), ('runs', 'O'), (',', 'O'), ('eight', 'O'), ('hits', 'O'), ('and', 'O'), ('one', 'O'), ('walk', 'O'), ('with', 'O'), ('five', 'O'), ('strikeouts', 'O'), ('over', 'O'), ('seven', 'O')]"


## Tests

The tests below to ensure the core DataBlock code above works for **all** pretrained token classification models available in Hugging Face.  These tests are excluded from the CI workflow because of how long they would take to run and the amount of data that would be required to download.

**Note**: Feel free to modify the code below to test whatever pretrained classification models you are working with ... and if any of your pretrained token classification models fail, please submit a github issue *(or a PR if you'd like to fix it yourself)*

In [ ]:
# hide
[model_type for model_type in BLURR.get_models(task="TokenClassification") if (not model_type.startswith("TF"))]



['AlbertForTokenClassification',
 'BertForTokenClassification',
 'BigBirdForTokenClassification',
 'CamembertForTokenClassification',
 'CanineForTokenClassification',
 'ConvBertForTokenClassification',
 'DebertaForTokenClassification',
 'DebertaV2ForTokenClassification',
 'DistilBertForTokenClassification',
 'ElectraForTokenClassification',
 'FNetForTokenClassification',
 'FlaubertForTokenClassification',
 'FunnelForTokenClassification',
 'GPT2ForTokenClassification',
 'IBertForTokenClassification',
 'LayoutLMForTokenClassification',
 'LayoutLMv2ForTokenClassification',
 'LongformerForTokenClassification',
 'MPNetForTokenClassification',
 'MegatronBertForTokenClassification',
 'MobileBertForTokenClassification',
 'RemBertForTokenClassification',
 'RoFormerForTokenClassification',
 'RobertaForTokenClassification',
 'SqueezeBertForTokenClassification',
 'XLMForTokenClassification',
 'XLMRobertaForTokenClassification',
 'XLNetForTokenClassification']

In [ ]:
# hide
pretrained_model_names = [
    # "hf-internal-testing/tiny-albert",
    # "hf-internal-testing/tiny-bert",
    # "google/bigbird-roberta-base",
    # "camembert-base",
    # "google/canine-s",                                  # word_ids
    # "YituTech/conv-bert-base",
    # "hf-internal-testing/tiny-deberta",
    "microsoft/deberta-v2-xlarge",                      # word_ids
    # "sshleifer/tiny-distilbert-base-cased",
    # "hf-internal-testing/tiny-electra",
    # # "google/fnet-base",                               # forward() got an unexpected keyword argument 'output_attentions'
    # "flaubert/flaubert_small_cased",                    # word_ids 
    # "huggingface/funnel-small-base",
    # "sshleifer/tiny-gpt2",
    # "hf-internal-testing/tiny-layoutlm",
    # "allenai/longformer-base-4096",
    # "microsoft/mpnet-base",
    # "kssteven/ibert-roberta-base",
    # # "nvidia/megatron-bert-cased-345m",                # could not test           
    # "google/mobilebert-uncased",
    # 'google/rembert',
    # "junnyu/roformer_chinese_sim_char_ft_small",                 
    # "roberta-base",
    # "squeezebert/squeezebert-uncased",
    # "xlm-mlm-en-2048",                                  # word_ids
    # "xlm-roberta-base",
    # "xlnet-base-cased",
]


In [ ]:
raw_datasets = load_dataset("conll2003")
conll2003_df = pd.DataFrame(raw_datasets["train"])

labels = raw_datasets["train"].features["ner_tags"].feature.names

Reusing dataset conll2003 (/home/wgilliam/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6)


In [ ]:
# hide
model_cls = AutoModelForTokenClassification
bsz = 2
seq_sz = 128

test_results = []
for model_name in pretrained_model_names:
    error = None

    print(f"=== {model_name} ===\n")

    tok_kwargs = {"add_prefix_space": True} if 'deberta' in model_name else {}

    hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(model_name, model_cls=model_cls, tokenizer_kwargs=tok_kwargs)
    print(f"architecture:\t{hf_arch}\ntokenizer:\t{type(hf_tokenizer).__name__}\n")

    # not all architectures include a native pad_token (e.g., gpt2, ctrl, etc...), so we add one here
    if hf_tokenizer.pad_token is None:
        hf_tokenizer.add_special_tokens({"pad_token": "<pad>"})
        hf_config.pad_token_id = hf_tokenizer.get_vocab()["<pad>"]
        hf_model.resize_token_embeddings(len(hf_tokenizer))

  
    def get_x(r):
        if hf_arch == "canine":
            return [f"{word} " for word in r.tokens]
        else:
            return r.tokens

    
    before_batch_tfm = HF_TokenClassBeforeBatchTransform(hf_arch, hf_config, hf_tokenizer, hf_model, padding="max_length", max_length=seq_sz)
    blocks = (HF_TextBlock(before_batch_tfm=before_batch_tfm, input_return_type=HF_TokenClassInput), HF_TokenCategoryBlock(vocab=labels))
    dblock = DataBlock(blocks=blocks, get_x=get_x, get_y=ColReader("ner_tags"), splitter=RandomSplitter())

    dls = dblock.dataloaders(conll2003_df, bs=bsz)
    b = dls.one_batch()

    print("*** TESTING DataLoaders ***\n")
    test_eq(len(b), 2)
    test_eq(len(b[0]["input_ids"]), bsz)
    test_eq(b[0]["input_ids"].shape, torch.Size([bsz, seq_sz]))
    test_eq(len(b[1]), bsz)

    if hasattr(hf_tokenizer, "add_prefix_space"):
        test_eq(hf_tokenizer.add_prefix_space, True)
    
    test_results.append((hf_arch, type(hf_tokenizer).__name__, model_name, "PASSED", ""))
    dls.show_batch(dataloaders=dls, max_n=2, trunc_at=20)

    # except Exception as err:
    #     test_results.append((hf_arch, type(hf_tokenizer).__name__, model_name, "FAILED", err))



=== microsoft/deberta-v2-xlarge ===

architecture:	deberta_v2
tokenizer:	DebertaV2Tokenizer

*** TESTING DataLoaders ***



,word / target label
0,"[('MARKET', 'O'), ('TALK', 'O'), ('-', 'O'), ('USDA', 'B-ORG'), ('net', 'O'), ('change', 'O'), ('in', 'O'), ('weekly', 'O'), ('export', 'O'), ('commitments', 'O'), ('for', 'O'), ('the', 'O'), ('week', 'O'), ('ended', 'O'), ('August', 'O'), ('22', 'O'), (',', 'O'), ('includes', 'O'), ('old', 'O'), ('crop', 'O')]"
1,"[('The', 'O'), ('new', 'O'), ('company', 'O'), ('is', 'O'), ('called', 'O'), ('AdOn', 'B-ORG'), ('GmbH', 'I-ORG'), ('and', 'O'), ('is', 'O'), ('located', 'O'), ('in', 'O'), ('Hamburg', 'B-LOC'), (',', 'O'), ('Bernd', 'B-PER'), ('Schiphorst', 'I-PER'), (',', 'O'), ('president', 'O'), ('and', 'O'), ('chief', 'O'), ('operating', 'O')]"


In [ ]:
# hide_input
test_results_df = pd.DataFrame(test_results, columns=["arch", "tokenizer", "model_name", "result", "error"])
display_df(test_results_df)


## Summary

This module includes all the low, mid, and high-level API bits for token classification tasks data prep.

In [ ]:
# hide
from nbdev.export import notebook2script

notebook2script()


In [ ]:
# from transformers import CanineTokenizer, CanineForTokenClassification
# import torch

# tokenizer = CanineTokenizer.from_pretrained('google/canine-s')
# model = CanineForTokenClassification.from_pretrained('google/canine-s')

from transformers import XLMTokenizer, XLMForTokenClassification
import torch

tokenizer = XLMTokenizer.from_pretrained('xlm-mlm-en-2048')
model = XLMForTokenClassification.from_pretrained('xlm-mlm-en-2048')

inputs = tokenizer(['4.', 'Theybers', '25'], return_tensors="pt", is_split_into_words=True)
labels = torch.tensor([1] * inputs["input_ids"].size(1)).unsqueeze(0) # Batch size 1

outputs = model(**inputs, labels=labels)
loss = outputs.loss
logits = outputs.logits

In [ ]:
inputs["input_ids"]


In [ ]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])


In [ ]:
tokenizer.convert_tokens_to_ids(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0]))

In [ ]:
tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0]))

In [ ]:
tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=True).split()